In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from add_transformation import add_transformation
from datasets import load_dataset
# import fugashi
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import libvoikko
import numpy as np
import ast

In [14]:
from Language import Language
from Pipeline import Pipeline

ImportError: cannot import name 'Pipeline' from 'Pipeline' (/Users/christianjensen/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Uni/3 B1 - NLP/NLP-2022/Pipeline.py)

### Coding guidelines

1. Export whole functions to external python files


In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/christianjensen/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
languages = {
    'english': Language(
        name = 'english',
        tokenizer = word_tokenize,
        cleaner = lambda text: text.lower()
        ),
    'japanese': Language(
        name = 'japanese',
        tokenizer = lambda text: text.split(), #[word.surface for word in fugashi.Tagger()(text)],
        cleaner = lambda text: text.lower()
        ),
    'finnish': Language(
        name = 'finnish',
        tokenizer = word_tokenize,
        cleaner = lambda text: text.lower()
        ),
}

In [8]:
def is_valid_language(x):
    return x['language'] in languages

train_set = train_set.filter(is_valid_language)
validation_set = validation_set.filter(is_valid_language)

100%|██████████| 14/14 [00:00<00:00, 79.32ba/s]


In [34]:
train_set = train_set.map(add_transformation)
validation_set = validation_set.map(add_transformation)

  0%|          | 0/29868 [00:00<?, ?ex/s]


NameError: name 'languages' is not defined

In [8]:
train_set[14]

{'question_text': 'Montako kuuta Saturnuksella on?',
 'document_title': 'Saturnuksen kuut',
 'language': 'finnish',
 'annotations': {'answer_start': [25], 'answer_text': ['62']},
 'document_plaintext': '\nSaturnukselta tunnetaan 62 kuuta (vuonna 2012).[1] Kuiden kokonaismäärää ei tulla koskaan tarkoin määrittelemään, koska renkaiden jääkappaleet ovat kaikki teknisesti kuita, ja on vaikea vetää rajaa pikkukuun ja suuren renkaankappaleen välille. Saturnuksen tunnetuimmat ja suurimmat kuut ovat sisimmästä uloimpaan: Atlas, Prometheus, Pandora, Epimetheus, Janus, Mimas, Enceladus, Tethys, Telesto, Calypso, Dione, Helene, Rhea, Titan, Hyperion, Japetus ja Phoebe.',
 'document_url': 'https://fi.wikipedia.org/wiki/Saturnuksen%20kuut',
 'is_answerable': 1,
 'tokenized_question': ['montako', 'kuuta', 'saturnuksella', 'on', '?']}

In [9]:
for language in languages.keys():
    count_words = lambda text: np.unique(text, return_counts=True) # Count occurences of words in text
    sort_words = lambda word_count: np.argsort(-word_count[1]) # Get list of sorted indices based on most frequent words
    zip_words = lambda word_counts, sort_indices: list(zip(word_counts[0][sort_indices],word_counts[1][sort_indices])) # Zip the most frequent words with its number of occurences
    def find_most_common(text):
        """Finds the most frequent words in a text together with its number of occurences"""
        word_count = count_words(text)
        return zip_words(word_count, sort_words(word_count))

    filtered = train_set.filter(lambda x: x['language'] == language)['tokenized_question']
    first = [sublist[0] for sublist in filtered]
    last = [sublist[-1] for sublist in filtered]
    
    print(f"""
    Language: {language}
    Most frequent first word:
    {find_most_common(first)}
    Most frequent last word:
    {find_most_common(last)}\n\n
    """)

100%|██████████| 30/30 [00:01<00:00, 27.09ba/s]

    Language: english
    Most frequent first word:
    [('when', 2244), ('what', 2105), ('how', 1306), ('who', 1060), ('where', 488), ('which', 66), ('why', 42), ('in', 26), ('on', 12), ('at', 10), ('is', 8), ('does', 4), ('by', 4), ('from', 2), ('was', 2), ('for', 2), ('whats', 2), ('do', 2), ('king', 2), ('are', 2)]
    Most frequent last word:
    [('?', 7379), ("''", 2), ('\\', 2), ('bce', 2), ('metabolite', 2), ('zombie', 2)]


    
100%|██████████| 30/30 [00:02<00:00, 12.89ba/s]

    Language: japanese
    Most frequent first word:
    [('日本', 392), ('『', 306), ('アメリカ', 106), ('世界', 94), ('第', 56), ('「', 50), ('ドイツ', 46), ('ジョージ', 44), ('ジョン', 44), ('ウィリアム', 44), ('初代', 36), ('カール', 32), ('ニューヨーク', 32), ('映画', 30), ('イタリア', 28), ('サー', 26), ('イギリス', 24), ('漫画', 24), ('中国', 24), ('ロバート', 24), ('国際', 22), ('リチャード', 20), ('チャールズ', 20), ('自動', 20), ('東京', 20), ('ドラえもん', 18), ('ロシア', 18), ('フランス', 18), ('阪神', 18), ('大', 18), ('コソボ', 18

In [10]:
english = train_set.filter(lambda x: x['language'] == 'english')

100%|██████████| 30/30 [00:01<00:00, 29.59ba/s]


In [11]:
train_set['tokenized_question']

[['milloin', 'charles', 'fort', 'syntyi', '?'],
 ['“',
  'ダン',
  '”',
  'ダニエル',
  '・',
  'ジャド',
  'ソン',
  '・',
  'キャラハン',
  'の',
  '出身',
  'は',
  'どこ'],
 ['mitä', 'on', 'altruismi', '?'],
 ['mikä', 'oli', 'wilhelm', 'wagner', 'viimeinen', 'sävellys', '?'],
 ['missä', 'harz', 'sijaitsee', '?'],
 ['miten', 'paljon', 'täysikasvuinen', 'ahma', 'painaa', '?'],
 ['サー', '・', 'マイケル', '・', 'フィリップ', '・', 'ジャガー', 'の', '出身', 'は', '？'],
 ['milloin', 'leonid', 'brežnev', 'kuoli', '?'],
 ['when', 'was', 'quantum', 'field', 'theory', 'developed', '?'],
 ['who',
  'was',
  'the',
  'first',
  'nobel',
  'prize',
  'winner',
  'for',
  'literature',
  '?'],
 ['ラオス', '王国', 'の', '王族', 'の', '名前', 'は', 'なん', 'です', 'か', '？'],
 ['kuka', 'keksi', 'releen', '?'],
 ['日本',
  'テレビ',
  '系列',
  '『',
  'zip',
  '!',
  '』',
  'の',
  '初代',
  'の',
  '司会',
  'は',
  '誰',
  'です',
  'か',
  '？'],
 ['kuka', 'frank', 'mir', 'on', '?'],
 ['montako', 'kuuta', 'saturnuksella', 'on', '?'],
 ['西村',
  '眞悟',
  '弁護',
  '士',
  '法',
  '

In [25]:
# Avoid tokenizing again
vectorizer = CountVectorizer(
        tokenizer= lambda x: x,
        preprocessor=lambda x: x,
)
X = vectorizer.fit_transform(english['tokenized_question'])

In [26]:
# Set one label manually to zero to contain both classes
Y = english['is_answerable']
Y[100] = 0

In [27]:
english[0]

{'question_text': 'When was quantum field theory developed?',
 'document_title': 'Quantum field theory',
 'language': 'english',
 'annotations': {'answer_start': [159], 'answer_text': ['1920s']},
 'document_plaintext': 'Quantum field theory naturally began with the study of electromagnetic interactions, as the electromagnetic field was the only known classical field as of the 1920s.[8]:1',
 'document_url': 'https://en.wikipedia.org/wiki/Quantum%20field%20theory',
 'is_answerable': 1,
 'tokenized_question': ['when',
  'was',
  'quantum',
  'field',
  'theory',
  'developed',
  '?']}

In [28]:
clf = LogisticRegression(random_state=0).fit(X, Y)

In [29]:
clf.score(X, Y)

0.9998646636892679

In [30]:
validationX = vectorizer.transform(validation_set['tokenized_question'])
validationY = validation_set['is_answerable']

In [31]:
clf.score(validationX, validationY)

1.0

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=64025ca8-12b9-4f73-a13d-23a3d5b6f385' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>